In [2]:
import os
import h5py
import numpy as np
# import matplotlib.pyplot as plt
from functions.preprocess import input_shaping, split_index
# from functions.decoders import CNNDecoder
from functions.metrics import compute_rmse, compute_pearson
from functions.channel_mapping import channel_mapping
from tensorflow.keras import initializers
from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.layers import Dense, Activation, Lambda , Input , Flatten ,Conv2D, MaxPooling2D, Dropout 
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization,MaxPooling2D,Dropout, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D,Lambda
from tensorflow.keras.optimizers import Adam
from functions.metrics import compute_rmse, compute_pearson, pearson_r
from keras.initializers import glorot_uniform
from keras.utils import np_utils
from tensorflow import random
import time as timer
from keras.models import load_model
import keras.backend as K
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print(tf.executing_eagerly())


True


In [3]:
print('The tensorflow version is:')
print(tf.__version__)
print('If the version is > 2.6 STMCubeAI import from keras will return an error, please then use the tflite version')
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


The tensorflow version is:
2.6.0
If the version is > 2.6 STMCubeAI import from keras will return an error, please then use the tflite version
Num GPUs Available:  1
Default GPU Device: /device:GPU:0


In [10]:
def identity_block(X, f, filters, stage, block):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters

    X_shortcut = X
   
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    
    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation('relu')(X)

    return X

def custom_residual_block(X,feature,block='0',stride=[],kernel=3,identity=True,batch_norm_end=False):
    conv_name_base = 'res'  + block + '_branch'
    bn_name_base = 'bn' + block + '_branch'
    X_shortcut = X
    no_stride=False
    if not stride:
        stride = [1]*len(feature)
        no_stride = True
        
    # if no_stride:
    #     pad='same'
    # else:
    #     pad='valid' 

    for i in range(0,len(feature)):

        if i == 0:
            X = Conv2D(filters=feature[i], kernel_size=(kernel, kernel), strides=(stride[i], stride[i]),kernel_initializer=glorot_uniform(seed=0), padding='same', name=conv_name_base + str(i))(X)
        else:
            X = Conv2D(filters=feature[i], kernel_size=(kernel, kernel), strides=(1,1), padding='same',kernel_initializer=glorot_uniform(seed=0), name=conv_name_base + str(i))(X)    
        if not batch_norm_end:
            X=BatchNormalization(axis=3, name=bn_name_base + str(i))(X)

    if not identity:
        output = X.shape
        output = output[-2:]
        output = tf.convert_to_tensor(output)
        proto_tensor = tf.make_tensor_proto(output) 
        output = tf.make_ndarray(proto_tensor) 
        input = X_shortcut.shape
        input = input[-2:]
        input = tf.convert_to_tensor(input)
        proto_tensor = tf.make_tensor_proto(input)  
        input = tf.make_ndarray(proto_tensor) 
        if output[-1]==input[-1] and output[-2]==input[-2]:  
            X_shortcut = Conv2D(filters=feature[-1], kernel_size=(kernel, kernel), strides=(1, 1), padding='same',kernel_initializer=glorot_uniform(seed=0), name=conv_name_base + 'short')(X)
        else:
            
            kernel_eq = input[-2] - ((output[-2]-1)*stride[0])
            X_shortcut = Conv2D(filters=feature[-1], kernel_size=(kernel_eq, kernel_eq), strides=(stride[0], stride[0]),kernel_initializer=glorot_uniform(seed=0), padding='same', name=conv_name_base + 'short')(X)
        if not batch_norm_end:
            X_shortcut = BatchNormalization(axis=3,name=bn_name_base + 'short')(X_shortcut)
    #X = Add()([X, X_shortcut])
    if batch_norm_end:
        X = BatchNormalization(axis=3,name=bn_name_base + 'final')(X)
    X = Activation('relu')(X)                       

    return X

def convolutional_block(X, f, filters, stage, block, s=2):
   
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters
    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def ResNet50_micro(input_shape=(10, 10, 1)):

    X_input = Input(input_shape)

    #X = ZeroPadding2D((3, 3))(X_input)

    X = Conv2D(64, (2, 2), strides=(1, 1),padding='same', name='conv1')(X_input)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    #X = MaxPooling2D((3, 3), strides=(1, 1))(X)

    X = convolutional_block(X, f=3, filters=[4, 4, 16], stage=2, block='a', s=1)
    X = identity_block(X, 3, [4, 4, 16], stage=2, block='b')
    X = identity_block(X, 3, [4, 4, 16], stage=2, block='c')


    X = convolutional_block(X, f=3, filters=[8, 8, 32], stage=3, block='a', s=1)
    X = identity_block(X, 3, [8, 8, 32], stage=3, block='b')
    X = identity_block(X, 3, [8, 8, 32], stage=3, block='c')
    X = identity_block(X, 3, [8, 8, 32], stage=3, block='d')

    X = convolutional_block(X, f=3, filters = [16, 16, 64], stage=4, block='a', s=1)
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='b')
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='c')
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='d')
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='e')
    X = identity_block(X, 3, [16, 16, 64], stage=4, block='f')

    X = X = convolutional_block(X, f=3, filters=[32, 32, 128], stage=5, block='a', s=1)
    X = identity_block(X, 3, filters=[32, 32, 128], stage=5, block='b')
    X = identity_block(X, 3, filters=[32, 32, 128], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2))(X)
    X = Flatten()(X)
    X = Dense(100)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dense(10)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dense(2)(X)
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

def custom_ResNet_micro(featurses,strides=[],input_shape=(10, 10, 1),name='custom_resnet'):

    X_input = Input(input_shape)
    X = Conv2D(16,kernel_size= (3, 3), strides=(1, 1),padding='same',kernel_initializer=glorot_uniform(seed=0), name='conv1')(X_input)
    #X = BatchNormalization(axis=3, name='bn_conv1')(X)
    for i in range(len(featurses)):
        if i == 0:
            X= custom_residual_block(X,feature=featurses[i], block=str(i))
        else:        
            X= custom_residual_block(X,feature=featurses[i],block=str(i))

    X = AveragePooling2D(pool_size=(2, 2))(X)
    X = Flatten()(X)
    # X = Dense(50,activation='relu',kernel_initializer=glorot_uniform(seed=0))(X)
    # X = Dropout(0.30)(X)
    X = Dense(5,kernel_initializer=glorot_uniform(seed=0))(X)
    X = Activation('relu')(X)
    X = Dropout(0.10)(X)
    X = Dense(2,kernel_initializer=glorot_uniform(seed=0))(X)
    model = Model(inputs=X_input, outputs=X, name=name)

    return model

In [19]:

seed = 100 # random seed for reproducibility

print ("Starting simulation")
run_start = timer.time()

feature_list = ['sua_rate', 'mua_rate']
feature = feature_list[1] # select which spike feature: SUA=0, MUA=1

# specify filename to be processed (choose from the list available at https://zenodo.org/record/583331)
file_name = 'indy_20160915_01'          # file name
kinematic_folder = 'kinematic_data/'    # kinematic data folder
feature_folder = 'spike_data/features/' # spike features folder
result_folder = 'results/'              # results folder
wdw_time = 0.128 # window size in second
lag = -32 # lag between kinematic and feature data (minus indicate feature lagging behaind kinematic)
delta_time = 0.004 # sampling interval in second
wdw_samp = int(round(wdw_time/delta_time))
ol_samp = wdw_samp-1

# open spike features from hdf5 file
feature_file = feature_folder+file_name+'_spike_features_'+str(int(wdw_time*1e3))+'ms.h5'
print ("Loading input features from file: "+feature_file)
with h5py.File(feature_file,'r') as f:
    input_feature = f[feature][()]
channel_mapping_file ='raw_data/indy_20160915_01.nwb' #r'F:/dropbox/Dropbox (Imperial NGNI)/NGNI Share/Workspace/Zheng/Research_Topics/signal processing plantform/prediction/decoding/raw_data/indy_20170127_03.nwb'

with h5py.File(channel_mapping_file, "r") as f:
    channel_loc = f['/general/extracellular_ephys/electrode_map'][()]
input_feature = channel_mapping(input_feature,channel_loc)
# open kinematic data from hdf5 file
kinematic_file = kinematic_folder+file_name+'_kinematic_data.h5'
print ("Loading kinematic data from file: "+kinematic_file)
with h5py.File(kinematic_file,'r') as f:
    cursor_vel = f['cursor_vel'][()] # in mm/s

# set QRNN hyperparameters
units = 300 # SUA: 200, MUA: 150
window_size = 2
epochs = 32
batch_size = 64
dropout = 0.
lrate = 0.0003

num_fold = 10 # number of folds
 # SUA: 0.002, MUA: 0.0035 
weight_decay = 1e-4
print("Hyperparameters >> units={}, window_size={}, epochs={}, batch_size={}, dropout={:.1f}, lrate={:.4f}".format(
    units, window_size, epochs, batch_size, dropout, lrate))          

# Define dictionary of parameters    
num_layers = 1 # number of layers
optimizer = 'adam' # optimizer
timesteps = 1 # number of timesteps (lag + current)

input_dim = timesteps#input_feature.shape[1] # input dimension
output_dim = cursor_vel.shape[1] # output dimension
verbose = 0

setting1 = 2
setting2 =3
# in_planes = {1:4,2:8,3:16,4:32}

featureses = {0:[16,16]}#,1:[16,16]}#,2:[16,16]} 
num_blockses = {1:[1],2:[1,1],3:[1,1,1],4:[2],5:[2,2],6:[2,2,2],7:[3],8:[3,3],9:[3,3,3]}

features = [16,16,32,64]
num_blocks = [3,3,3]
model_name='resnet'
save_path = result_folder+file_name+'_'+model_name+optimizer+'_batch_size_{}_'.format(batch_size)+'_lr_{}_'.format(lrate)+'_'+feature+'_'+str(int(wdw_time*1e3))+'/'


Starting simulation
Loading input features from file: spike_data/features/indy_20160915_01_spike_features_128ms.h5
Loading kinematic data from file: kinematic_data/indy_20160915_01_kinematic_data.h5
Hyperparameters >> units=300, window_size=2, epochs=32, batch_size=64, dropout=0.0, lrate=0.0003


In [20]:
# initialise performance scores (RMSE and CC) with nan values

loss_train = np.full((num_fold, epochs), np.nan)
loss_valid = np.copy(loss_train)
rmse_valid = np.full((num_fold, output_dim), np.nan)
rmse_test = np.copy(rmse_valid)
cc_valid = np.copy(rmse_valid)
cc_test = np.copy(rmse_valid)
time_train = np.full((num_fold), np.nan)
time_test = np.copy(time_train) 

print ("Formatting input feature data")
tstep = timesteps # timestep (lag + current) samples
stride = 1 # number of samples to be skipped
X_in = input_shaping(input_feature.reshape(input_feature.shape[0],-1), timesteps, 1)
X_in = X_in.reshape(X_in.shape[0],timesteps,10,10)
 
print ("Formatting output (kinematic) data")
diff_samp = cursor_vel.shape[0]-X_in.shape[0]
Y_out = cursor_vel[diff_samp:,:] # in mm/s (remove it for new corrected velocity)

print ("Splitting input dataset into training, validation, and testing subdataset")
all_train_idx, all_valid_idx, all_test_idx = split_index(Y_out, num_fold)
global temp
for i in range(num_fold): 
    train_idx = all_train_idx[i]
    valid_idx = all_valid_idx[i]
    test_idx = all_test_idx[i]
    
    # specify training dataset
    X_train = X_in[train_idx,:]            
    Y_train = Y_out[train_idx,:]
    
    # specify validation dataset
    X_valid = X_in[valid_idx,:]
    Y_valid = Y_out[valid_idx,:]
    
    # specify validation dataset
    X_test = X_in[test_idx,:]
    Y_test = Y_out[test_idx,:]
    
    epsilon = 1e-4
    # Standardize (z-score) input dataset
    X_train_mean = np.nanmean(X_train, axis=0)
    X_train_std = np.nanstd(X_train, axis=0) 
    X_train = (X_train - X_train_mean)/(X_train_std+epsilon)
    X_valid = (X_valid - X_train_mean)/(X_train_std +epsilon)
    X_test = (X_test - X_train_mean)/(X_train_std +epsilon)
    
    # Zero mean (centering) output dataset
    Y_train_mean = np.nanmean(Y_train, axis=0)
    Y_train_std = np.nanstd(Y_train, axis=0) 
    Y_train = (Y_train - Y_train_mean)/(Y_train_std+epsilon)
    Y_valid = (Y_valid - Y_train_mean)/(Y_train_std +epsilon)
    Y_test = (Y_test - Y_train_mean)/(Y_train_std +epsilon)
    # Y_train = Y_train - Y_train_mean 
    # Y_valid = Y_valid - Y_train_mean
    # Y_test = Y_test - Y_train_mean
           
    #Re-align data to take lag into accountc
    if lag < 0:
        X_train = X_train[:lag,:] # remove lag first from end (X lag behind Y)
        Y_train = Y_train[-lag:,:] # reomve lag first from beginning
        X_valid = X_valid[:lag,:]
        Y_valid = Y_valid[-lag:,:]
        X_test = X_test[:lag,:]
        Y_test = Y_test[-lag:,:]
    if lag > 0:
        X_train = X_train[lag:,:] # reomve lag first from beginning
        Y_train = Y_train[:-lag,:] # remove lag first from end (X lead in front of Y)
        X_valid = X_valid[lag:,:]
        Y_valid = Y_valid[:-lag,:]            
        X_test = X_test[lag:,:]
        Y_test = Y_test[:-lag,:]
        
    # set seed to get reproducible results
    np.random.seed(seed)
    random.set_seed(seed)
    X_train = np.moveaxis(X_train,1,-1)
    X_valid = np.moveaxis(X_valid,1,-1)
    X_test = np.moveaxis(X_test,1,-1)

# SMALL_SIZE = 9
# MEDIUM_SIZE = 14
# BIGGER_SIZE = 18

# plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
# plt.rc('axes', titlesize=14)     # fontsize of the axes title
# plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
# plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
# plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
# plt.rc('legend', fontsize=11)    # legend fontsize
# plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# N_rows=10
# N_cols=10
# start_val = 0 # pick an element for the code to plot the following N**2 values
# fig, axes = plt.subplots(N_rows,N_cols)
# fig.set_size_inches(15,19)
# items = list(range(0, 10))
# for row in range(N_rows):
#   for col in range(N_cols):
#     idx = start_val+col+N_rows*row
#     axes[row,col].imshow(X_train[idx], cmap='viridis')
#     #fig.subplots_adjust(hspace=0.25)
#     axes[row,col].set_title('sample {}'.format(idx))
#     axes[row,col].set_xticks([])
#     axes[row,col].set_yticks([])
# fig.savefig('images.jpg',format="jpg",dpi=600, bbox_inches='tight', pad_inches=0.1)
# fig_vel, axes_vel =plt.subplots(1)
# fig_vel.set_size_inches(15,5)  
# X = np.arange(0,100,1,dtype=int)   
# axes_vel.xaxis.grid(True, linestyle='--')
# axes_vel.yaxis.grid(True, linestyle='--')
# axes_vel.plot(X,Y_train[0:100,0],linewidth=2,label="X axis velocity")
# axes_vel.plot(X,Y_train[0:100,1],linewidth=2,label="y axis velocity")
# axes_vel.set(xlabel='sample index',title='velocity graph')
# axes_vel.legend(loc='upper right')
# fig_vel.savefig('velocity.jpg',format="jpg",dpi=600, bbox_inches='tight', pad_inches=0.1)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_pearson_r', patience=5,restore_best_weights=True)
model = custom_ResNet_micro(featurses=featureses)
#model = ResNet50_micro()
opt = Adam(learning_rate=1e-3)
model.compile(loss='mse', optimizer=opt,metrics=[pearson_r])
model.fit(X_train, Y_train,validation_data=(X_valid,Y_valid) ,epochs=10, batch_size=64,verbose=True,shuffle=True,callbacks=callback)



# Y_valid_predict = model.predict(X_valid)
# start = timer.time()
# Y_test_predict = model.predict(X_test)
# end = timer.time()


# print("Model testing took {:.2f} seconds".format(end - start)) 
# time_test[i] = end - start

# # Compute performance metrics    
# rmse_vld = compute_rmse(Y_valid, Y_valid_predict)
# rmse_tst = compute_rmse(Y_test, Y_test_predict)
# cc_vld = compute_pearson(Y_valid, Y_valid_predict)
# cc_tst = compute_pearson(Y_test, Y_test_predict)

# rmse_valid[i,:] = rmse_vld
# rmse_test[i,:] = rmse_tst
# cc_valid[i,:] = cc_vld
# cc_test[i,:] = cc_tst
# if os.path.exists(save_path) == False:
#     os.makedirs(save_path)
    
# print("Fold-{} | Validation RMSE: {:.4f}".format(i, np.nanmean(rmse_vld)))
# print("Fold-{} | Validation CC: {:.4f}".format(i, np.nanmean(cc_vld)))
# print("Fold-{} | Testing RMSE: {:.4f}".format(i, np.nanmean(rmse_tst)))
# print("Fold-{} | Testing CC: {:.4f}".format(i, np.nanmean(cc_tst)))

# run_end = timer.time()
# mean_rmse_valid = np.nanmean(rmse_valid, axis=0) 
# mean_rmse_test = np.nanmean(rmse_test, axis=0)
# mean_cc_valid = np.nanmean(cc_valid, axis=0)
# mean_cc_test = np.nanmean(cc_test, axis=0)
# mean_time =  np.nanmean(time_train, axis=0)
# print("----------------------------------------------------------------------")
# print("Validation Mean RMSE: %.5f " %(np.mean(mean_rmse_valid)))
# print("Validation Mean CC: %.5f " %(np.mean(mean_cc_valid)))
# print("Testing Mean RMSE: %.5f " %(np.mean(mean_rmse_test)))
# print("Testing Mean CC: %.5f " %(np.mean(mean_cc_test)))
# print("----------------------------------------------------------------------")

# # storing evaluation results into hdf5 file
# result_filename = result_folder+file_name+'_resnet50f10_'+feature+'_'+str(int(wdw_time*1e3))+'ms.h5'
# print ("Storing results into file: "+result_filename)
# with h5py.File(result_filename,'w') as f:
#     f['Y_true'] = Y_test
#     f['Y_predict'] = Y_test_predict
#     f['rmse_valid'] = rmse_valid
#     f['rmse_test'] = rmse_test
#     f['cc_valid'] = cc_valid
#     f['cc_test'] = cc_test
#     f['time_train'] = time_train
#     f['time_test'] = time_test

# run_time = run_end - run_start
# print ("Finished whole processes within %.2f seconds" % run_time)
# model.summary()

Formatting input feature data
Formatting output (kinematic) data
Splitting input dataset into training, validation, and testing subdataset
Epoch 1/10
1191/1191 [==============================] - 10s 8ms/step - loss: 0.6306 - pearson_r: -0.1832 - val_loss: 0.5257 - val_pearson_r: -0.3516
Epoch 2/10
1191/1191 [==============================] - 9s 7ms/step - loss: 0.5421 - pearson_r: -0.2199 - val_loss: 0.5058 - val_pearson_r: -0.3411
Epoch 3/10
1191/1191 [==============================] - 9s 8ms/step - loss: 0.5230 - pearson_r: -0.2287 - val_loss: 0.5145 - val_pearson_r: -0.3126
Epoch 4/10
1191/1191 [==============================] - 10s 8ms/step - loss: 0.5139 - pearson_r: -0.2291 - val_loss: 0.5276 - val_pearson_r: -0.3170
Epoch 5/10
1191/1191 [==============================] - 10s 8ms/step - loss: 0.4972 - pearson_r: -0.2281 - val_loss: 0.5437 - val_pearson_r: -0.3031
Epoch 6/10
1191/1191 [==============================] - 9s 7ms/step - loss: 0.4901 - pearson_r: -0.2296 - val_loss: 0.

In [21]:
# model.summary()
micro_valid = np.ones((1,10,10,1))
output_micro = model.predict(micro_valid)
print(output_micro)

[[ 0.25227815 -0.74588186]]


In [22]:

model.save('cnnD.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.representative_dataset = representative_dataset
# # Ensure that if any ops can't be quantized, the converter throws an error
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# # Set the input and output tensors to uint8 (APIs added in r2.3)
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8
tflite_model = converter.convert()
with open('cnnD.tflite', 'wb') as f:
  f.write(tflite_model)